In [1]:
from model_builder.GIS import tinyGIS
TG = tinyGIS()
import pandas as pd
import numpy as np
from typing import Dict, Union
from itertools import product
import os
import xarray as xr

tinyGIS initiated


# Real case

## preparation of modfied soil files with dimension n, subbasin, and dimension s as soil

In [2]:
# read the soil file, get it majority and save
file_path = './data/gis/'
file_name_csv = 'West_stats_soil_classes.csv'
file_name_nc = 'West_stats_soil_classes.nc'

soil_type = pd.read_csv(file_path+file_name_csv)
soil_name = ['1 clay', '2 silty clay', '3 sandy clay', '4 clay loam','5 silty clay loam',\
             '6 sandy clay loam', '7 loam', '8 silty loam', '9 sandy loam', '10 silt',\
             '11 loamy sand', '12 sand']

# drop the 0 soil type which is mostly to represent water
soil_type = soil_type.drop(columns = 'frac_0')
soil_type = soil_type.drop(columns= 'Unnamed: 0')

# get the majority of the soil type for each subbasin
soil_type   =  TG.manipulating_fractions(soil_type,
                                         df_mapping={'prefix': 'frac_'},
                                         action = 'majority')

# save the modified file into the csv
soil_type.to_csv(file_path+'modified_'+file_name_csv)

## xarray object and netcdf
# set index to of COMID and name it n
soil_type = soil_type.set_index('COMID')
soil_type.index.name = 'n'

# Convert majority to xarray DataArray
data_array_majority = xr.DataArray(soil_type['majority'].values, dims=('n',), coords={'n': soil_type.index})

# create xarray for majority and frac_
data_array_frac = xr.DataArray(soil_type.filter(like='frac').values, dims=('n', 's'), coords={'n': soil_type.index, 's': soil_name})

# Combine into xarray Dataset
ds = xr.Dataset({'soil_majority': data_array_majority, 'soil_frac': data_array_frac})

if os.path.isfile(file_path+'modified_'+file_name_nc):
    os.remove(file_path+'modified_'+file_name_nc)
ds.to_netcdf(file_path+'modified_'+file_name_nc)

ds

<xarray.Dataset>
Dimensions:        (n: 453, s: 12)
Coordinates:
  * n              (n) int64 71022153 71022160 71022164 ... 71039250 71039266
  * s              (s) <U17 '1 clay' '2 silty clay' ... '12 sand'
Data variables:
    soil_majority  (n) int64 3 3 3 3 3 3 3 3 3 3 3 3 ... 3 3 3 3 3 3 3 3 3 3 3 3
    soil_frac      (n, s) float64 0.0 0.0 1.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

## preparation of modfied soil files with dimension n, subbasin, and l as land cover

In [3]:
# read the soil file, get it majority and save
file_path = './data/gis/'
file_name_csv = 'West_stats_NA_NALCMS_landcover_2020_30m.csv'
file_name_nc = 'West_stats_NA_NALCMS_landcover_2020_30m.nc'


land_cover = pd.read_csv(file_path+file_name_csv)
LULC_name =  ['1 Temperate or sub-polar needleleaf forest',\
              '2 Sub-polar taiga needleleaf forest',\
              '3 Tropical or sub-tropical broadleaf evergreen forest',\
              '4 Tropical or sub-tropical broadleaf deciduous forest',\
              '5 Temperate or sub-polar broadleaf deciduous forest',\
              '6 Mixed forest',\
              '7 Tropical or sub-tropical shrubland',\
              '8 Temperate or sub-polar shrubland',\
              '9 Tropical or sub-tropical grassland',\
              '10 Temperate or sub-polar grassland',\
              '11 Sub-polar or polar shrubland-lichen-moss',\
              '12 Sub-polar or polar grassland-lichen-moss',\
              '13 Sub-polar or polar barren-lichen-moss',\
              '14 Wetland',\
              '15 Cropland',\
              '16 Barren lands',\
              '17 Urban',\
              '18 Water',\
              '19 Snow and Ice']

# smooth land cover and remove below 5% land cover and renormalize
land_cover =  TG.manipulating_fractions(land_cover,
                                         df_mapping={'prefix': 'frac_'},
                                         action = 'normalize',
                                         minimum_value = 0.05)

# get the majority of the soil type for each subbasin
land_cover   =  TG.report_max_frac(land_cover,
                                   df_mapping={'prefix': 'frac_'})


# save the modified file into the csv
land_cover.to_csv(file_path+'modified_'+file_name_csv)

# set index to of COMID and name it n
land_cover = land_cover.set_index('COMID')
land_cover.index.name = 'n'

# Convert majority to xarray DataArray
data_array_majority = xr.DataArray(land_cover['majority'].values, dims=('n',), coords={'n': land_cover.index})

# create xarray for majority and frac_
data_array_frac = xr.DataArray(land_cover.filter(like='frac').values, dims=('n', 'l'), coords={'n': land_cover.index, 'l': LULC_name})

# Combine into xarray Dataset
ds = xr.Dataset({'LULC_majority': data_array_majority, 'LULC_frac': data_array_frac})


if os.path.isfile(file_path+'modified_'+file_name_nc):
    os.remove(file_path+'modified_'+file_name_nc)
ds.to_netcdf(file_path+'modified_'+file_name_nc)

ds

<xarray.Dataset>
Dimensions:        (n: 453, l: 19)
Coordinates:
  * n              (n) int64 71022153 71022160 71022164 ... 71039250 71039266
  * l              (l) <U53 '1 Temperate or sub-polar needleleaf forest' ... ...
Data variables:
    LULC_majority  (n) object '15' '15' '15' '15' '15' ... '16' '16' '1' '16'
    LULC_frac      (n, l) float64 0.0 0.0 0.0 0.0 0.0 ... 0.4622 0.0 0.1151 0.0

## preparation of modfied soil files with dimension n, subbasin, and m as combination of soil and land cover

In [10]:
# call the function to create the merger of soil and land cover
path = './data/gis/'
file_name_nc = 'modified_West_Combination.nc'
soil_type = pd.read_csv('./data/gis/modified_West_stats_soil_classes.csv')
land_cover = pd.read_csv('./data/gis/modified_West_stats_NA_NALCMS_landcover_2020_30m.csv')

result,report, ds =  TG.intersect_df(soil_type, land_cover,
                                     df_mappings={'df1': {'id': 'COMID', 'prefix':'frac_' , 'data_name':'soil'}, 
                                                  'df2': {'id': 'COMID', 'prefix':'frac_' , 'data_name':'LULC'}},
                                     remove_zero_combinations = True)

# get the majority of cobination for each subbasin
result   =  TG.report_max_frac(result,
                               df_mapping={'prefix': 'comb_'})

# rename the index for the result
result.index.name = 'n'

# create xarray for frac_
data_array_frac = xr.DataArray(result.filter(like='comb').values, dims=('n', 'm'), coords={'n': result.index, 'm': result.filter(like='comb').columns})

# create xarray for majority
data_array_majority = xr.DataArray(result['majority'].values, dims=('n'), coords={'n': result.index})

# Combine into xarray Dataset
ds = xr.Dataset({'comb_frac': data_array_frac, 'comb_majority': data_array_majority})


# convert the value using the m
report = report.set_index('Combinations')
report.index.name = 'm'
ds_report = report.to_xarray()

ds['comb_soil'] = xr.DataArray(ds_report['soil'].values, dims=('m'))
ds['comb_LULC'] = xr.DataArray(ds_report['LULC'].values, dims=('m'))

if os.path.isfile(path+file_name_nc):
    os.remove(path+file_name_nc)
ds.to_netcdf(path+file_name_nc)

ds

The indexes of all DataFrames are exactly the same with the same order.
total number of non zero combinations:  15


<xarray.Dataset>
Dimensions:        (n: 453, m: 15)
Coordinates:
  * n              (n) int64 71022153 71022160 71022164 ... 71039250 71039266
  * m              (m) object 'comb_0001' 'comb_0002' ... 'comb_0015'
Data variables:
    comb_frac      (n, m) float64 0.0 0.0 0.0 0.07281 0.0 ... 0.0 0.0 0.0 0.0
    comb_majority  (n) object '0007' '0007' '0007' ... '0008' '0001' '0008'
    comb_soil      (m) object '3' '3' '3' '3' '3' '3' ... '3' '8' '8' '8' '8'
    comb_LULC      (m) object '1' '5' '6' '8' '10' ... '19' '1' '10' '16' '19'

## mean elevation based on dimension n

In [8]:
## add other information to ds
file_path = './data/gis/'
file_name_csv = 'West_stats_elv.csv'
file_name_nc = 'West_stats_elv.nc'

elev = pd.read_csv(file_path+file_name_csv)
elev.index = elev['COMID'].astype(int)
elev['mean_elev'] = elev['mean']
elev = elev.drop(columns=['COMID','min','max','mean','median'])


elev.index.name = 'n'
ds_elev = elev.to_xarray()
ds_elev['mean_elev'].attrs['unit'] = 'm'

if os.path.isfile(path+file_name_nc):
    os.remove(path+file_name_nc)
ds_elev.to_netcdf(path+file_name_nc)

ds_elev

<xarray.Dataset>
Dimensions:    (n: 453)
Coordinates:
  * n          (n) int64 71022153 71022160 71022164 ... 71039250 71039266
Data variables:
    mean_elev  (n) float64 893.2 915.9 912.3 ... 2.161e+03 1.913e+03 2.121e+03